In [1]:
import pandas as pd
from pathlib import Path

data_path = Path("../data/dataframe.csv")
print(data_path)

..\data\dataframe.csv


In [4]:
dataframe = pd.read_csv(data_path)

In [ ]:
dataframe

,tags,instruction,category,intent,response
0,BCIPZ,"I would like to acivate a card, can you help me?",CARD,activate_card,I'm here to assist you with that! Activating y...
1,BCILZ,"I have to activate an Visa online, how can I d...",CARD,activate_card,I'm here to assist you with activating your {{...
2,BCIPQZ,I'd like to actiate a card where do i do it,CARD,activate_card,I can help you with that! Activating your card...
3,BCLPQZ,I'd likke to activate a visa on mobile i need ...,CARD,activate_card,I'm here to assist you with activating your {{...
4,BCILPZ,"I would ilke to activate a credit card online,...",CARD,activate_card,I'm here to assist you with activating your cr...
...,...,...,...,...,...
25540,BCEQZ,I'm trying to find apassword i need help,PASSWORD,set_up_password,I'm here to assist you in finding and setting ...
25541,BCIPQ,im looking for a password will uhelp me,PASSWORD,set_up_password,I'm here to assist you in setting up a passwor...
25542,BCIPZ,"I'd like to set up a password, will you help me?",PASSWORD,set_up_password,I'm here to assist you in setting up your pass...
25543,BCQ,wanna set up a password i need assistance,PASSWORD,set_up_password,I'm here to assist you in setting up your pass...


In [ ]:
dataframe['intent'].nunique()

26

In [ ]:
dataframe['category'].nunique()

9

In [6]:
from transformers.models.bart import BartForConditionalGeneration,BartTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [94]:
model_name = Path("../artifacts/spelling-correction-english-base")
model = BartForConditionalGeneration.from_pretrained(model_name).to(device) # type: ignore
tokenizer = BartTokenizer.from_pretrained(model_name)

In [95]:
def correct_grammar(texts, model, tokenizer, device):
    input_ids = tokenizer.batch_encode_plus(texts,return_tensors="pt",padding=True,truncation=True).to(device)
    outputs = model.generate(
        **input_ids,
        max_length=128,
        num_beams=5,
        early_stopping=True,
    )
    corrected_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return corrected_text



In [45]:
sample_text = dataframe['instruction'][:128].to_list()

In [46]:
sample_text

['I would like to acivate a card, can you help me?',
 'I have to activate an Visa online, how can I do it?',
 "I'd like to actiate a card where do i do it",
 "I'd likke to activate a visa on mobile i need assistance",
 'I would ilke to activate a credit card online, will you help me?',
 'activate a cfredit card',
 'I received a new card and I want to activate it, will oyu help me?',
 'id like to activate a visa online',
 'I got to activaste a card, could I get some help?',
 'I have to activatean Amex on mobile, will you help me?',
 "I'd like to activate a visa on mobile where can i do it",
 'i have tyo activate a visa can ya help me',
 'help me activate an Amex online',
 "I'd like to activate a visa online",
 'I need to activate a Master Card online, could I get some help?',
 "I'd like to activate a credit card online were could i do it",
 'id like to activate a master card online could uhelp me',
 "I'd like to activate a Visa onmobile, where can I do it?",
 'I got to activate a Master

In [47]:
correct_grammar(sample_text,model,tokenizer,device)

['I would like to activate a card, can you help me?',
 'I have to activate an Visa online, how can I do it?',
 "I'd like to activate a card where do I do it?",
 "I'd like to activate a visa on mobile, I need assistance.",
 'I would like to activate a credit card online, and will you help me?',
 ' activate a credit card.',
 'I received a new card and I want to activate it, will you help me?',
 "I'd like to activate a visa online.",
 'I got to activate a card, could I get some help?',
 'I have to activate an app on mobile, how will you help me?',
 "I'd like to activate a visa on mobile, where can I do it?",
 'I have to activate a visa can ya help me.',
 'help me activate an Amazon online.',
 "I'd like to activate a visa online.",
 'I need to activate a Master Card online, and could I get some help?',
 "I'd like to activate a credit card online where could I do it?",
 "I'd like to activate a master card online could help me.",
 "I'd like to activate a Visa on mobile, where can I do it?",


In [48]:
(25500/128)*3.5

697.265625

In [52]:
import tqdm
corrected_texts = []
texts = dataframe['instruction'].to_list()
batch_size = 128
def batchify(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

for batch_text in tqdm.tqdm(batchify(texts,batch_size)):
    corrected_text = correct_grammar(batch_text,model,tokenizer,device)
    corrected_texts.extend(corrected_text)

200it [24:22,  7.31s/it]


In [50]:
len(texts)

25545

In [53]:
corrected_texts

['I would like to activate a card, can you help me?',
 'I have to activate an Visa online, how can I do it?',
 "I'd like to activate a card where do I do it?",
 "I'd like to activate a visa on mobile, I need assistance.",
 'I would like to activate a credit card online, and will you help me?',
 ' activate a credit card.',
 'I received a new card and I want to activate it, will you help me?',
 "I'd like to activate a visa online.",
 'I got to activate a card, could I get some help?',
 'I have to activate an app on mobile, how will you help me?',
 "I'd like to activate a visa on mobile, where can I do it?",
 'I have to activate a visa can ya help me.',
 'help me activate an Amazon online.',
 "I'd like to activate a visa online.",
 'I need to activate a Master Card online, and could I get some help?',
 "I'd like to activate a credit card online where could I do it?",
 "I'd like to activate a master card online could help me.",
 "I'd like to activate a Visa on mobile, where can I do it?",


In [54]:
len(corrected_texts)

25545

In [55]:
len(texts)

25545

In [56]:
dataframe['instruction_corrected'] = corrected_texts

In [57]:
dataframe.head()

,tags,instruction,category,intent,response,instruction_corrected
0,BCIPZ,"I would like to acivate a card, can you help me?",CARD,activate_card,I'm here to assist you with that! Activating y...,"I would like to activate a card, can you help me?"
1,BCILZ,"I have to activate an Visa online, how can I d...",CARD,activate_card,I'm here to assist you with activating your {{...,"I have to activate an Visa online, how can I d..."
2,BCIPQZ,I'd like to actiate a card where do i do it,CARD,activate_card,I can help you with that! Activating your card...,I'd like to activate a card where do I do it?
3,BCLPQZ,I'd likke to activate a visa on mobile i need ...,CARD,activate_card,I'm here to assist you with activating your {{...,"I'd like to activate a visa on mobile, I need ..."
4,BCILPZ,"I would ilke to activate a credit card online,...",CARD,activate_card,I'm here to assist you with activating your cr...,"I would like to activate a credit card online,..."


In [59]:
dataframe_save_path = Path("../data/corrected_dataframe.csv")
dataframe.to_csv(dataframe_save_path,index=False)

In [2]:
dataframe_save_path = Path("../data/corrected_dataframe.csv")
dataframe_new = pd.read_csv(dataframe_save_path)

In [3]:
dataframe_new.head()

,tags,instruction,category,intent,response,instruction_corrected
0,BCIPZ,"I would like to acivate a card, can you help me?",CARD,activate_card,I'm here to assist you with that! Activating y...,"I would like to activate a card, can you help me?"
1,BCILZ,"I have to activate an Visa online, how can I d...",CARD,activate_card,I'm here to assist you with activating your {{...,"I have to activate an Visa online, how can I d..."
2,BCIPQZ,I'd like to actiate a card where do i do it,CARD,activate_card,I can help you with that! Activating your card...,I'd like to activate a card where do I do it?
3,BCLPQZ,I'd likke to activate a visa on mobile i need ...,CARD,activate_card,I'm here to assist you with activating your {{...,"I'd like to activate a visa on mobile, I need ..."
4,BCILPZ,"I would ilke to activate a credit card online,...",CARD,activate_card,I'm here to assist you with activating your cr...,"I would like to activate a credit card online,..."


In [91]:
for category in dataframe_new['category'].unique():
    print(f"\n🔹 Category: {category}")
    
    sub_df = dataframe_new[dataframe_new['category'] == category]
    
    for intent in sub_df['intent'].unique():
        intent_df = sub_df[sub_df['intent'] == intent]
        example = intent_df.iloc[1]  # İlk örnek
        print(f"  - Intent: {intent}")
        print(f"    👉 Instruction: {example['instruction_corrected']}")


🔹 Category: CARD
  - Intent: activate_card
    👉 Instruction: I have to activate an Visa online, how can I do it?
  - Intent: activate_card_international_usage
    👉 Instruction: I'm traveling overseas, I need to activate a visa for international usage.
  - Intent: block_card
    👉 Instruction: My name has been stolen, I need help locking it.
  - Intent: cancel_card
    👉 Instruction: I'd like to cancel a credit  card on mobile, could I get some help?
  - Intent: check_card_annual_fee
    👉 Instruction: I cannot find my card annual fees how could I check them?
  - Intent: check_current_balance_on_card
    👉 Instruction: I got to see the fucking current balance on my cards, how could I do it?

🔹 Category: LOAN
  - Intent: apply_for_loan
    👉 Instruction: I got to apply for a loan, how could I do it?
  - Intent: apply_for_mortgage
    👉 Instruction: I need a mortgage  can you help me applying for it?
  - Intent: cancel_loan
    👉 Instruction: Wanna cancel a loan where could I do it?
  

In [44]:
dataframe = pd.read_csv(Path("../data/corrected_dataframe.csv"))
dataframe.head()

,tags,instruction,category,intent,response,instruction_corrected
0,BCIPZ,"I would like to acivate a card, can you help me?",CARD,activate_card,I'm here to assist you with that! Activating y...,"I would like to activate a card, can you help me?"
1,BCILZ,"I have to activate an Visa online, how can I d...",CARD,activate_card,I'm here to assist you with activating your {{...,"I have to activate an Visa online, how can I d..."
2,BCIPQZ,I'd like to actiate a card where do i do it,CARD,activate_card,I can help you with that! Activating your card...,I'd like to activate a card where do I do it?
3,BCLPQZ,I'd likke to activate a visa on mobile i need ...,CARD,activate_card,I'm here to assist you with activating your {{...,"I'd like to activate a visa on mobile, I need ..."
4,BCILPZ,"I would ilke to activate a credit card online,...",CARD,activate_card,I'm here to assist you with activating your cr...,"I would like to activate a credit card online,..."


In [45]:
inputs = dataframe['instruction_corrected']
targets = dataframe['intent']
inputs

0        I would like to activate a card, can you help me?
1        I have to activate an Visa online, how can I d...
2            I'd like to activate a card where do I do it?
3        I'd like to activate a visa on mobile, I need ...
4        I would like to activate a credit card online,...
                               ...                        
25540          I'm trying to find a passport, I need help.
25541             I'm looking for a password will help me.
25542     I'd like to set up a password, will you help me?
25543           Wanna set up a password I need assistance.
25544      I got to set up a appssword, could you help me?
Name: instruction_corrected, Length: 25545, dtype: object

In [46]:
targets

0          activate_card
1          activate_card
2          activate_card
3          activate_card
4          activate_card
              ...       
25540    set_up_password
25541    set_up_password
25542    set_up_password
25543    set_up_password
25544    set_up_password
Name: intent, Length: 25545, dtype: object

In [47]:
targets.value_counts()

intent
activate_card                        1000
find_branch                          1000
human_agent                           999
check_recent_transactions             999
close_account                         999
customer_service                      998
block_card                            998
find_ATM                              998
apply_for_mortgage                    997
make_transfer                         997
activate_card_international_usage     997
check_card_annual_fee                 996
cancel_card                           996
cancel_mortgage                       996
create_account                        996
cancel_transfer                       995
recover_swallowed_card                994
check_current_balance_on_card         993
apply_for_loan                        992
check_mortgage_payments               990
cancel_loan                           990
dispute_ATM_withdrawal                989
check_loan_payments                   989
check_fees                 

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report,accuracy_score


In [49]:
train_df, test_df = train_test_split(
    dataframe[['instruction_corrected','intent']],
    test_size=0.2,
    random_state=42,
    stratify=dataframe[['instruction_corrected','intent']]['intent']
)

In [50]:
test_df['intent'].value_counts()

intent
find_ATM                             200
find_branch                          200
close_account                        200
check_recent_transactions            200
human_agent                          200
activate_card                        200
block_card                           200
customer_service                     200
check_card_annual_fee                199
apply_for_mortgage                   199
activate_card_international_usage    199
make_transfer                        199
recover_swallowed_card               199
cancel_mortgage                      199
create_account                       199
cancel_card                          199
check_current_balance_on_card        199
cancel_transfer                      199
cancel_loan                          198
check_loan_payments                  198
dispute_ATM_withdrawal               198
apply_for_loan                       198
check_mortgage_payments              198
check_fees                           189
set_up_pa

In [51]:
x_train,y_train = train_df['instruction_corrected'],train_df['intent']
x_test,y_test = test_df['instruction_corrected'],test_df['intent']

In [52]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000, ngram_range=(1,2), stop_words='english')),
    ('clf', RandomForestClassifier(n_estimators=200,criterion="gini"))
])
pipeline.fit(x_train,y_train)
pipeline.score(x_test,y_test)

0.9933450773145429

In [57]:
import pickle
import os

pipe_folder_path = os.path.join("..","artifacts","classifier")
pipe_save_path = os.path.join(pipe_folder_path,"classifier.pkl")
os.makedirs(pipe_folder_path,exist_ok=True)
with open(pipe_save_path, 'wb') as file:
    pickle.dump(pipeline, file)

In [58]:
with open(pipe_save_path,"rb") as pipe:
    pipeline = pickle.load(pipe)

In [59]:
pipeline.predict(["Could you help me with activating my card?"])

array(['activate_card'], dtype=object)

In [53]:
pipeline.predict(['could you help me with activate my card'])

array(['activate_card'], dtype=object)

In [54]:
pipeline.predict(["could you connect me fucking operator"])

array(['human_agent'], dtype=object)

In [55]:
text = "i would like to cancel amortgage, could you helpme with th is"
pipeline.predict([text])

array(['cancel_card'], dtype=object)

In [56]:
corrected_text = correct_grammar([text],model,tokenizer,device)

NameError: name 'correct_grammar' is not defined

In [97]:
corrected_text

['I would like to cancel a mortgage, could you help me with this?']

In [98]:
pipeline.predict(corrected_text)

array(['cancel_mortgage'], dtype=object)

In [104]:
example = dataframe.iloc[0]
text = example['instruction_corrected']
doc = example['response']

In [13]:
def count_words(row):
    row['response_word_number'] = len(row['response'].split())
    return row
dataframe_new = dataframe_new.apply(count_words,axis=1)

In [17]:
dataframe_new['response_word_number'].max()

430

In [105]:
text

'I would like to activate a card, can you help me?'

In [78]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

class Embedder:
    def __init__(self,model_path:Path | str,device:str):
        self.model = AutoModel.from_pretrained(model_path).to(device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
    
    def embed(self,texts):
        encoded_input = self.tokenizer(texts,padding=True,truncation=True,return_tensors="pt").to(self.model.device)
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings

    def mean_pooling(self,model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

embedder = Embedder(Path("../artifacts/all-MiniLM-L6-v2"),device="cuda")

In [5]:
import numpy as np
import tqdm
def get_embeddings(texts, batch_size=32):
    embeddings = []
    for i in tqdm.tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i + batch_size]
        embeddings.append(embedder.embed(batch).cpu().numpy())
    
    return np.vstack(embeddings)  # (N, 384)

In [6]:
response_embeddings = get_embeddings(texts=dataframe_new['response'].to_list(),batch_size=128)

100%|██████████| 200/200 [01:07<00:00,  2.95it/s]


In [51]:
response_embeddings.shape

(25545, 384)

In [7]:
from collections import defaultdict

category_to_embeddings = defaultdict(list)
category_to_texts = defaultdict(list)

categories = dataframe_new['intent'].to_list()
sentences = dataframe_new['response'].to_list()

for emb, cat, text in zip(response_embeddings, categories, sentences):
    category_to_embeddings[cat].append(emb)
    category_to_texts[cat].append(text)

In [8]:
import faiss

category_to_faiss = {}
category_to_id_map = {}

for cat, embs in category_to_embeddings.items():
    emb_array = np.array(embs).astype('float32')
    
    # FAISS index (cosine similarity için normalize edilmiş dot product)
    faiss.normalize_L2(emb_array)
    index = faiss.IndexFlatIP(emb_array.shape[1])
    index.add(emb_array)
    
    category_to_faiss[cat] = index
    category_to_id_map[cat] = category_to_texts[cat]

In [64]:
category_to_faiss.keys()

dict_keys(['activate_card', 'activate_card_international_usage', 'apply_for_loan', 'apply_for_mortgage', 'block_card', 'cancel_card', 'cancel_loan', 'cancel_mortgage', 'cancel_transfer', 'check_card_annual_fee', 'check_current_balance_on_card', 'check_fees', 'check_loan_payments', 'check_mortgage_payments', 'check_recent_transactions', 'close_account', 'create_account', 'customer_service', 'dispute_ATM_withdrawal', 'find_ATM', 'find_branch', 'get_password', 'human_agent', 'make_transfer', 'recover_swallowed_card', 'set_up_password'])

In [67]:
index_path = Path("../indexes")

os.makedirs(index_path,exist_ok=True)

for intent,index in category_to_faiss.items():
    intent_path = os.path.join(index_path,intent)
    os.makedirs(intent_path)
    faiss.write_index(index,os.path.join(intent_path,"index.index"))

for intent,docs in category_to_id_map.items():
    intent_path = os.path.join(index_path,intent)
    os.makedirs(intent_path,exist_ok=True)
    with open(os.path.join(intent_path,"docs.pkl"),"wb") as fp:
        pickle.dump(docs,fp)
print("indexes and docs saved.")


indexes and docs saved.


In [23]:
query = "How do I change my password?"

intent = "get_password"
query_embedding = embedder.embed(query).cpu().numpy()
index = category_to_faiss[intent]
D, I = index.search(query_embedding, k=3)

In [24]:
top_texts = [category_to_id_map[intent][i] for i in I[0]]

In [25]:
for text in top_texts:
    print(text)

I'm here to assist you in getting a new password. To ensure the security of your account, please follow these steps:

1. Go to the login page of our website or application.
2. Click on the "Forgot Password" or "Reset Password" option.
3. You'll be prompted to enter your registered email address or username associated with your account.
4. Once you've entered the required information, click on the "Reset" or "Submit" button.
5. You should receive an email with further instructions and a link to reset your password.
6. Open the email and click on the password reset link provided.
7. Follow the instructions on the password reset page to create a new password.
8. Make sure to choose a strong, unique password to enhance the security of your account.
9. After setting the new password, log in to your account using the updated credentials.

If you encounter any difficulties during this process or have any additional questions, feel free to reach out. We're here to help you regain access to you

In [18]:
I

array([[179, 189, 694]], dtype=int64)

In [19]:
D

array([[0.71618116, 0.70803094, 0.694358  ]], dtype=float32)

In [14]:
unique_categories = set(category for category in categories)

In [15]:
unique_categories

{'activate_card',
 'activate_card_international_usage',
 'apply_for_loan',
 'apply_for_mortgage',
 'block_card',
 'cancel_card',
 'cancel_loan',
 'cancel_mortgage',
 'cancel_transfer',
 'check_card_annual_fee',
 'check_current_balance_on_card',
 'check_fees',
 'check_loan_payments',
 'check_mortgage_payments',
 'check_recent_transactions',
 'close_account',
 'create_account',
 'customer_service',
 'dispute_ATM_withdrawal',
 'find_ATM',
 'find_branch',
 'get_password',
 'human_agent',
 'make_transfer',
 'recover_swallowed_card',
 'set_up_password'}

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch
llm_path = Path("../artifacts/Qwen3-0.6B")

llm_tokenizer = AutoTokenizer.from_pretrained(llm_path)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_path,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

In [38]:
def build_rag_prompt(query, retrieved_docs):
    context = "\n".join([f"- {doc}" for doc in retrieved_docs])
    prompt = f"""You are a helpful banking customer relationship assistant. Use the following documents to answer the question.

Context:
{context}

Question: {query}
Answer:"""
    messages = [{"role": "user", "content": prompt}]
    text = llm_tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)
    return text

In [29]:
streamer = TextStreamer(llm_tokenizer, skip_prompt=True, skip_special_tokens=True)

In [34]:
def stream_answer(prompt, max_new_tokens=1024):
    input_ids = llm_tokenizer(prompt, return_tensors="pt").to(llm_model.device)
    _ = llm_model.generate(
        **input_ids,
        max_new_tokens=max_new_tokens,
        streamer=streamer,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        repetition_penalty=1.1
    )

In [39]:
prompt = build_rag_prompt(query,retrieved_docs=top_texts)

In [40]:
stream_answer(prompt)

To change your password, follow these steps:

1. Go to the login page on our website.
2. Look for the "Forgot Password" or "Reset Password" option.
3. Click on it, which redirects you to the password reset page.
4. Enter your email address associated with your account.
5. Check your email inbox for a password reset link.
6. Click on the link provided in the email to reset your password.
7. Create a new password that includes a combination of letters, numbers, and special characters.
8. Confirm the password by re-entering it.
9. Save the changes and use your new password to log in to your account.


In [76]:
from transformers.models.bart import BartForConditionalGeneration,BartTokenizer
from typing import List
class Correcter:
    def __init__(self,path:Path|str,device:str = "cuda"):
        self.model = BartForConditionalGeneration.from_pretrained(path).to(device)
        self.tokenizer = BartTokenizer.from_pretrained(path)
    
    def correct(self,texts:List[str]):
        assert isinstance(texts,list), "Texts parameter should be a list!"
        input_ids = self.tokenizer.batch_encode_plus(texts,return_tensors="pt",padding=True,truncation=True).to(self.model.device)
        outputs = self.model.generate(
            **input_ids,
            max_length=128,
            num_beams=5,
            early_stopping=True,
        )
        corrected_text = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return corrected_text

correcter = Correcter(r"C:\Users\ömer\Desktop\crm-rag-llm\artifacts\correcter")



In [77]:
correcter.correct(["Th isis a texxxt"])

['This is a text.']

In [82]:
from faiss.loader import read_index_binary

In [87]:
index_path = Path("../indexes/activate_card/index.index")
index = faiss.read_index(str(index_path))

In [79]:
from transformers import TextIteratorStreamer
import threading

class LLM:
    def __init__(self,path,device):
        self.tokenizer = AutoTokenizer.from_pretrained(path)
        self.model = AutoModelForCausalLM.from_pretrained(
            path,
            device_map=device,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
        )
        self.max_new_tokens = 1024
    
    def build_rag_prompt(self,prompt,retrieved_docs):
        
        context = "\n".join([f"- {doc}" for doc in retrieved_docs])
        prompt = f"""You are a helpful banking customer relationship assistant. Use the following documents to answer the question. AND DO NOT FORGET TO BE POLITE.

    Context:
    {context}

    Question: {query}
    Answer:"""
        messages = [{"role": "user", "content": prompt}]
        text = llm_tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False 
        )
        return text
    
    def generate_and_stream(self,query,retrieved_docs):
        prompt = self.build_rag_prompt(query,retrieved_docs)
        inputs = self.tokenizer(prompt,return_tensors="pt").to(self.model.device)
        streamer = TextIteratorStreamer(self.tokenizer, skip_prompt=True, skip_special_tokens=True)
        generation_kwargs = dict(
            **inputs,
            streamer=streamer,
            max_new_tokens=self.max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
        )
        thread = threading.Thread(target=self.model.generate, kwargs=generation_kwargs)
        thread.start()
        for token in streamer:
            yield token


In [ ]:
class FullPipeline:
    def __init__(
            self,
            artifacts_path:Path,
            indexes_path:Path,
            device
    ):
        correcter_path = os.path.join(artifacts_path,"correcter")
        self.correcter = Correcter(correcter_path)
        classifier_path = os.path.join(artifacts_path,"classifier")
        with open(os.path.join(classifier_path,"classifier.pkl"),"rb") as pipe_fp:
            self.classifier = pickle.load(pipe_fp)
        
        embedder_path = os.path.join(artifacts_path,"embedder")
        self.embedder = Embedder(embedder_path,device=device)

        llm_path = os.path.join(artifacts_path,"llm")
        self.llm = LLM(llm_path,device)
        self.indexes_path = indexes_path

    def pipe(self,text):

        corrected_text = self.correcter.correct([text])
        print("---Corrected text:",corrected_text)
        intent = self.classifier.predict(corrected_text)[0]
        print("---intent:",intent)
        index = faiss.read_index(os.path.join(self.indexes_path,intent,"index.index"))
        docs_path = os.path.join(self.indexes_path,intent,"docs.pkl")
        with open(docs_path,"rb") as docs_fp:
            docs = pickle.load(docs_fp)
        
        corrected_text_embeds = self.embedder.embed(corrected_text).cpu().numpy()
        _, I = index.search(corrected_text_embeds, k=3)
        retrieved_docs = [docs[i] for i in I[0]]
        return self.llm.generate_and_stream(corrected_text,retrieved_docs)



        

In [104]:
artifacts_path = Path("../artifacts")
indexes_path = Path("../indexes")
device = "cuda"

pipeline = FullPipeline(artifacts_path=artifacts_path,indexes_path=indexes_path,device=device)

In [105]:
for token in pipeline.pipe(query):
    print(token, end="", flush=True)

---Corrected text: ['What should I do for cancelling my card?']
---intent: cancel_card
To cancel your card, follow these steps:  

1. **Contact** your bank’s customer service at {{Customer Support Phone Number}} or visit the [[Company Website URL]].  
2. Provide them with your card details (e.g., card number) so they can help you proceed.  
3. Follow the instructions provided by the support team to complete the cancellation process.  

Let me know if you need further assistance!

In [106]:
query = "What sould i do for cancelling my cardd"
for token in pipeline.pipe(query):
    print(token,end="",flush=True)

---Corrected text: ['What should I do for cancelling my card?']
---intent: cancel_card
To cancel your card, follow these steps:

1. **Contact your card issuer’s customer service** either via phone at {{Customer Support Phone Number}} or through the official website.  
2. Provide them with your card details (e.g., card number, expiration date, CVV code) so they can help you complete the cancellation.  
3. Follow the instructions provided by the customer service team to cancel your card properly.  

Let me know if you need further assistance!

In [108]:
dataframe_new[dataframe_new['intent'] == "dispute_ATM_withdrawal"]

,tags,instruction,category,intent,response,instruction_corrected
17868,BCIP,"I got to dispute an ATM withdrawal, could you ...",ATM,dispute_ATM_withdrawal,"It's our responsibility, and we're sorry for a...","I got to dispute an ATM withdrawal, could you ..."
17869,BCEI,"I do not recognise a withdrawal, what do I hav...",ATM,dispute_ATM_withdrawal,"We regret the inconvenience, and we're sorry f...","I do not recognise a withdrawal, what do I hav..."
17870,BCILPZ,"their is an ATM transaction I didn't make, wil...",ATM,dispute_ATM_withdrawal,I regret that any inconvenience this may have ...,"Their is an ATM transaction I didn't make, and..."
17871,BCLQZ,i dont recognise an atm trasaction i need assi...,ATM,dispute_ATM_withdrawal,I'm sorry to hear that you don't recognize an ...,"I don't recognise an atm transaction, I need a..."
17872,BCIPQ,wanna dispute a transaction where could i do it,ATM,dispute_ATM_withdrawal,I'm sorry to hear that you want to dispute a t...,We can dispute a transaction where could I do it?
...,...,...,...,...,...,...
18852,BCIPQZ,I'd like to dispute a withdrawsal how to do it,ATM,dispute_ATM_withdrawal,I'm sorry to hear that you want to dispute a w...,I'd like to dispute a withdrawal how to do it.
18853,BCI,"I want to dispute an ATM withdrawal, where do ...",ATM,dispute_ATM_withdrawal,I appreciate you reaching out to us regarding ...,"I want to dispute an ATM withdrawal, why do I ..."
18854,BCZ,"I want to dispute a withdrawal , help me",ATM,dispute_ATM_withdrawal,I'm sorry to hear that you want to dispute a w...,I want to dispute a withdrawal help me.
18855,BCQZ,idont recognise a transaction help me,ATM,dispute_ATM_withdrawal,I'm genuinely sorry for any confusion regardin...,Don't recognise a transaction help me.


In [110]:
query ="wanna dispuute a transactionaw where could i do it"
for token in pipeline.pipe(query):
    print(token,end="",flush=True)

---Corrected text: ['Wanna dispute a transaction where could I do it?']
---intent: dispute_ATM_withdrawal
To dispute a transaction, please contact our customer support team at [[Customer Support Phone Number]] or via the Live Chat on our website at [[Company Website URL]]. Ensure you have the necessary details (transaction date, amount, and supporting documents) before reaching out. We will investigate and resolve the dispute as efficiently as possible. Let me know if there’s anything else I can assist you with!